In [2]:
import pandas as pd

In [9]:
# read the names of companies from .csv (5rd col)
fortune500 = pd.read_csv('sp500_firm_execu.csv')
companies = fortune500.iloc[:,4]

# convert to list of strings and delete duplicates
companies = list(companies)
companies = list(dict.fromkeys(companies))

print(companies)

['AMERICAN AIRLINES GROUP INC', 'PINNACLE WEST CAPITAL CORP', 'ABBOTT LABORATORIES', 'ADVANCED MICRO DEVICES', 'AIR PRODUCTS & CHEMICALS INC', 'HONEYWELL INTERNATIONAL INC', 'SKYWORKS SOLUTIONS INC', 'ALCOA INC', 'HESS CORP', 'AMERICAN ELECTRIC POWER CO', 'AMERICAN EXPRESS CO', 'AFLAC INC', 'AMERICAN INTERNATIONAL GROUP', 'AMETEK INC', 'AMGEN INC', 'ANALOG DEVICES INC', 'APA CORP', 'APPLE INC', 'APPLIED MATERIALS INC', 'ARCHER-DANIELS-MIDLAND CO', 'AUTODESK INC', 'AUTOMATIC DATA PROCESSING', 'AVERY DENNISON CORP', 'BAKER HUGHES INC', 'BALL CORP', 'BANK OF NEW YORK MELLON CORP', 'BAXTER INTERNATIONAL INC', 'BECTON DICKINSON & CO', 'VERIZON COMMUNICATIONS INC', 'BERKSHIRE HATHAWAY', 'BEST BUY CO INC', 'BIO-RAD LABORATORIES INC', 'BOEING CO', 'ROBERT HALF INC', 'BRISTOL-MYERS SQUIBB CO', 'BROWN FORMAN CORP', 'CIGNA GROUP (THE)', 'CSX CORP', 'CAMPBELL SOUP CO', 'CONSTELLATION BRANDS', 'CARDINAL HEALTH INC', 'CATERPILLAR INC', 'JPMORGAN CHASE & CO', 'CHEVRON CORP', 'CHURCH & DWIGHT INC', 'C

In [ ]:
companies = companies[2:]

In [6]:
from datetime import date
from datetime import timedelta
    

class TimeFrame:
    def __init__(self, start: date, end: date):
        self.start = start
        self.end = end

    def __str__(self):
        return f"after:{self.start} before:{self.end}" # google format for dates
    
    def get_path(self):
        return f"from{self.start}to{self.end}"
    

# create a list of TimeFrame objects, every week going back 1 year
timeframes = []
for i in range(0, 52):
    end = date.today() - timedelta(weeks=i*1)
    start = end - timedelta(weeks=1)
    timeframes.append(TimeFrame(start, end))
    
for t in timeframes:
    print(t)


after:2025-02-15 before:2025-02-22
after:2025-02-08 before:2025-02-15
after:2025-02-01 before:2025-02-08
after:2025-01-25 before:2025-02-01
after:2025-01-18 before:2025-01-25
after:2025-01-11 before:2025-01-18
after:2025-01-04 before:2025-01-11
after:2024-12-28 before:2025-01-04
after:2024-12-21 before:2024-12-28
after:2024-12-14 before:2024-12-21
after:2024-12-07 before:2024-12-14
after:2024-11-30 before:2024-12-07
after:2024-11-23 before:2024-11-30
after:2024-11-16 before:2024-11-23
after:2024-11-09 before:2024-11-16
after:2024-11-02 before:2024-11-09
after:2024-10-26 before:2024-11-02
after:2024-10-19 before:2024-10-26
after:2024-10-12 before:2024-10-19
after:2024-10-05 before:2024-10-12
after:2024-09-28 before:2024-10-05
after:2024-09-21 before:2024-09-28
after:2024-09-14 before:2024-09-21
after:2024-09-07 before:2024-09-14
after:2024-08-31 before:2024-09-07
after:2024-08-24 before:2024-08-31
after:2024-08-17 before:2024-08-24
after:2024-08-10 before:2024-08-17
after:2024-08-03 bef

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

TIMEOUT = 5

for company in companies:
    print('Now on company:', company)
    for timeframe in timeframes:
        # Search google for news about the company, save top 10 results for each timeframe
        # the cols should be: date, title, url, snippet
        
        search_query = f'{company} news'
        search_url = f'https://www.google.com/search?q={search_query} {str(timeframe)}&tbm=nws'
        
        try:
            req = requests.get(search_url, timeout=TIMEOUT)
        except:
            print(f"Error with link {search_url}")
        if req.status_code != 200:
            print(f"Error with status code {req.status_code}")
            continue
        
        soup = BeautifulSoup(req.text, 'html5lib')
        
        

        links = soup.select('a:has(div.egMi0)')
        print(len(links))
        links = [link['href'].replace('/url?q=', '').split('&sa')[0] for link in links]
        
        # print the links
        print(links)
        
        
        
        
        dataframes = []
        for link in links:
            
            try:
                req = requests.get(link, timeout=TIMEOUT)
            except:
                print(f"Error with link {link}")
                continue
            
            if req.status_code != 200:
                print(f"Error with status code {req.status_code}")
                continue
            
            article_soup = BeautifulSoup(req.text, 'html5lib')
            
            # get the date
            try:
                article_date = article_soup.select_one('time').text if article_soup.select_one('time') is not None else None
                if article_date is None: 
                    article_date = article_soup.select_one('meta[itemprop="datePublished"]')['content'] if article_soup.select_one('meta[itemprop="datePublished"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="pubdate"]')['content'] if article_soup.select_one('meta[name="pubdate"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="date"]')['content'] if article_soup.select_one('meta[name="date"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="publishdate"]')['content'] if article_soup.select_one('meta[name="publishdate"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="timestamp"]')['content'] if article_soup.select_one('meta[name="timestamp"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="DC.date.issued"]')['content'] if article_soup.select_one('meta[name="DC.date.issued"]') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('meta[name="date"]')['content'] if article_soup.select_one('meta[name="date"]') is not None else None
                if article_date is None:
                    #check tags with class publish-date
                    article_date = article_soup.select_one('.publish-date').text if article_soup.select_one('.publish-date') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('.date').text if article_soup.select_one('.date') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('.timestamp').text if article_soup.select_one('.timestamp') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('.pubdate').text if article_soup.select_one('.pubdate') is not None else None
                if article_date is None:
                    article_date = article_soup.select_one('.date-published').text if article_soup.select_one('.date-published') is not None else None
                if article_date is None:
                    article_date = "Unknown"
            except:
                article_date = "Unknown"
                
            
            # get the title, try title tag first, then h1
            title = article_soup.select_one('title').text if article_soup.select_one('title') is not None else None
            if title is None:
                title = article_soup.select_one('h1').text if article_soup.select_one('h1') is not None else 'Unknown'
                
            # get the snippet from <p> tags
            snippet = ''
            for p in article_soup.select('p'):
                snippet += p.text + ' '
                
            # create a dataframe
            df = pd.DataFrame({'date': [article_date], 'title': [title], 'url': [link], 'snippet': [snippet]})
            dataframes.append(df)
            
            print(df)
        # save in /data/{company}/{timeframe}.csv
        # create dir if not exists
        if len(dataframes) == 0:
            continue
        os.makedirs(f'data/{company}', exist_ok=True)
        df = pd.concat(dataframes)
        
        # make the delimeter a rare symbol like §
        df.to_csv(f'data/{company}/{timeframe.get_path()}.csv', sep='§')
        
 
        



Now on company: PINNACLE WEST CAPITAL CORP
10
['https://ca.finance.yahoo.com/news/pinnacle-west-capital-post-q4-142200397.html', 'https://www.tipranks.com/news/company-announcements/pinnacle-west-capital-announces-executive-leadership-changes', 'https://www.marketscreener.com/quote/stock/PINNACLE-WEST-CAPITAL-COR-14077/news/Morgan-Stanley-Adjusts-Pinnacle-West-Capital-Price-Target-to-84-From-81-Maintains-Equal-Weight-Ra-49115248/', 'https://www.nasdaq.com/articles/pinnacle-west-capital-corp-pnw-shares-cross-4-yield-mark', 'https://www.inkl.com/news/what-are-wall-street-analysts-target-price-for-pinnacle-west-stock', 'https://www.zacks.com/stock/news/2418533/itron-gears-up-to-report-q4-earnings-heres-what-to-expect', 'https://www.businesswire.com/news/home/20250220250928/en/Ameriprise-Financial-Appoints-Glynis-A.-Bryan-to-Board-of-Directors', 'https://investorplace.com/market360/2025/02/20250219-blue-chip-upgrades-downgrades/', 'https://www.stocktitan.net/news/NSIT/ameriprise-financial-

KeyError: 'content'